In [1]:
import helpers.images
import pandas as pd

In [3]:
im_list = helpers.images.get_paeth_images('train')

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [01:49<00:00, 455.27it/s]


In [4]:
target = pd.read_csv('precomputed/train.csv')['PNG Size']

In [5]:
a_range = [0.25, 0.5, 1, 2, 4]
b_range = [0.25, 0.5, 1, 2, 4]

In [6]:
import helpers.measures
import numpy as np

d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:161: SyntaxWarning: invalid escape sequence '\e'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:167: SyntaxWarning: invalid escape sequence '\e'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:173: SyntaxWarning: invalid escape sequence '\e'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:179: SyntaxWarning: invalid escape sequence '\e'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:191: SyntaxWarning: invalid escape sequence '\k'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:216: SyntaxWarning: invalid escape sequence '\e'
  """
d:\CS\Sparse Models\PROJECT\Infer-PNG-Size\helpers\measures.py:222: SyntaxWarning: invalid escape sequence '\e'
  """


In [7]:
res = []
for a in a_range:
    for b in b_range:
        df = pd.DataFrame()
        df['PNG Size'] = target
        df['TANH'] = [helpers.measures.sparse_tanh(im, a, b) for im in im_list]
        print(a, b, np.array(df.corr())[0][1])
        res.append([a, b, np.array(df.corr())[0][1]])

0.25 0.25 0.4021564720139311
0.25 0.5 0.4252544572910366
0.25 1 0.4281303040548187
0.25 2 0.4251193649325857
0.25 4 0.4217809730726618
0.5 0.25 0.4150741714334675
0.5 0.5 0.42917600456428573
0.5 1 0.4309063991553721
0.5 2 0.43119776192536113
0.5 4 0.43025760729572116
1 0.25 0.42144931588814044
1 0.5 0.42692356976716894
1 1 0.4262439299022355
1 2 0.4141484063223477
1 4 0.3855891045276853
2 0.25 0.423764273509019
2 0.5 0.42241826525275045
2 1 0.4199507050254863
2 2 0.394106701618453
2 4 0.3798908315536628
4 0.25 0.4191036509040146
4 0.5 0.4173684269896649
4 1 0.4163290448956788
4 2 0.3747859547882099
4 4 nan


In [8]:
res = pd.DataFrame(res)
res.columns = ['A', 'B', 'cor']

In [9]:
import plotly.express as px
px.scatter_3d(res, x='A', y='B', z='cor')

In [10]:
res.sort_values(by='cor', ascending=False)

,A,B,cor
8,0.50,2.00,0.431198
7,0.50,1.00,0.430906
9,0.50,4.00,0.430258
6,0.50,0.50,0.429176
2,0.25,1.00,0.428130
11,1.00,0.50,0.426924
12,1.00,1.00,0.426244
1,0.25,0.50,0.425254
3,0.25,2.00,0.425119
15,2.00,0.25,0.423764


In [11]:
# BEST TANH : A=0.5 B=2

In [12]:
np.isnan(im_list).sum().sum()

0

In [13]:
(im_list[0] <= 0.25*255).mean() * len(im_list[0].flatten())

772.0

In [14]:
res = []
eps_range = [0.003, 0.005, 0.00625, 0.009, 0.0125]
for eps in eps_range:
    df = pd.DataFrame()
    df['PNG Size'] = target
    df['L0EPS'] = [helpers.measures.l0_epsilon(im, eps) for im in im_list]

    print(eps, np.array(df.corr())[0][1])
    res.append([eps, np.array(df.corr())[0][1]])

0.003 -0.4197740203438969
0.005 -0.43016118241443513
0.00625 -0.43016118241443513
0.009 -0.4259920959745827
0.0125 -0.4201763528387968


In [15]:
# BEST EPSILON = 0.005

In [16]:
res = []
p_range = [0.1, 0.25, 0.5, 1, 2, 3, 4, 6, 10]
for p in p_range:
    df = pd.DataFrame()
    df['PNG Size'] = target
    df['LP'] = [helpers.measures.lp_norm(im, p) for im in im_list]

    print(p, np.array(df.corr())[0][1])
    res.append([eps, np.array(df.corr())[0][1]])

0.1 0.07429174129386831
0.25 0.005281818627570614
0.5 -0.07765978885786048
1 -0.15828995455792197
2 0.4433512436313087
3 0.26539618531304837
4 0.428508445747196
6 0.38148870541991486
10 0.3371470304392979


In [17]:
# BEST P = 2

In [20]:
res = []
p_range = [0.1, 0.25, 0.5, 1, 2, 3, 4, 6, 10]
for p in p_range:
    df = pd.DataFrame()
    df['PNG Size'] = target
    df['LP-'] = [(1/(im**p).clip(min=1e-50)).mean() * len(im.flatten()) for im in im_list]

    print(p, np.array(df.corr())[0][1])
    res.append([eps, np.array(df.corr())[0][1]])

0.1 -0.4197740203438922
0.25 -0.4197740203438922
0.5 -0.4197740203438922
1 -0.4197740203438922
2 -0.4067217360503593
3 -0.39408175448352045
4 -0.3748012834907614
6 -0.3748012834907614
10 -0.3798968910508953


In [ ]:
# BEST = 0.5